# Granting Permissions

You must use `accountadmin` to grant and create roles.

In [ ]:
-- you must be accountadmin to create role and grant permissions
use role accountadmin;

In [ ]:
use schema p2p_demo.public;

-- Create a consumer role for users of the GDS application
CREATE ROLE IF NOT EXISTS gds_role;
GRANT APPLICATION ROLE se_snow_neo4j_graph_analytics.app_user TO ROLE gds_role;
-- Create a consumer role for administrators of the GDS application
CREATE ROLE IF NOT EXISTS gds_role;
GRANT APPLICATION ROLE se_snow_neo4j_graph_analytics.app_admin TO ROLE gds_role;

-- Grant access to consumer data
-- The application reads consumer data to build a graph object, and it also writes results into new tables.
-- We therefore need to grant the right permissions to give the application access.
GRANT USAGE ON DATABASE p2p_demo TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT USAGE ON SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;

-- required to read tabular data into a graph
GRANT SELECT ON ALL TABLES IN SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;
-- required to write computation results into a table
GRANT CREATE TABLE ON SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;
-- optional, ensuring the consumer role has access to tables created by the application
GRANT ALL PRIVILEGES ON FUTURE TABLES IN SCHEMA p2p_demo.public TO ROLE gds_role;
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA p2p_demo.public TO ROLE gds_role;
GRANT CREATE TABLE ON SCHEMA p2p_demo.public TO ROLE gds_role;



-- required to write computation results into a table
GRANT CREATE VIEW ON SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT CREATE VIEW ON SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT CREATE VIEW ON SCHEMA p2p_demo.public TO ROLE gds_role;

GRANT USAGE, MONITOR ON COMPUTE POOL NEO4J_GRAPH_DATA_SCIENCE_POOL_CPU_X64_XS TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT USAGE ON WAREHOUSE GDSONSNOWFLAKE TO APPLICATION se_snow_neo4j_graph_analytics;


Now we must switch to the role we created.

In [ ]:
use role gds_role;

Create Compute Pool

In [ ]:
CREATE COMPUTE POOL IF NOT EXISTS NEO4J_GRAPH_DATA_SCIENCE_POOL_CPU_X64_L
      MIN_NODES = 1
      MAX_NODES = 3
      INSTANCE_FAMILY = CPU_X64_L
      AUTO_RESUME = true
      AUTO_SUSPEND_SECS = 180
      INITIALLY_SUSPENDED = true;
    GRANT USAGE, MONITOR ON COMPUTE POOL NEO4J_GRAPH_DATA_SCIENCE_POOL_CPU_X64_L  TO APPLICATION se_snow_neo4j_graph_analytics;

## Aggregated Transactions

In [ ]:
CREATE OR REPLACE TABLE p2p_demo.public.P2P_AGG_TRANSACTIONS (
	SOURCENODEID NUMBER(38,0),
	TARGETNODEID NUMBER(38,0),
	TOTAL_AMOUNT FLOAT,
	TRANSACTION_COUNT FLOAT
) AS
SELECT sourceNodeId, targetNodeId, SUM(transaction_amount) AS total_amount, COUNT(*) AS transaction_count
FROM p2p_demo.public.P2P_TRANSACTIONS
GROUP BY sourceNodeId, targetNodeId;
SELECT * FROM p2p_demo.public.P2P_AGG_TRANSACTIONS;


CREATE OR REPLACE TABLE p2p_demo.public.P2P_AGG_TRANSACTIONS (
	SOURCENODEID NUMBER(38,0),
	TARGETNODEID NUMBER(38,0),
	TOTAL_AMOUNT FLOAT
) AS
SELECT sourceNodeId, targetNodeId, SUM(transaction_amount) AS total_amount
FROM p2p_demo.public.P2P_TRANSACTIONS
GROUP BY sourceNodeId, targetNodeId;
SELECT * FROM p2p_demo.public.P2P_AGG_TRANSACTIONS;

In [ ]:
GRANT SELECT ON ALL TABLES IN SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;

In [ ]:
USE DATABASE p2p_demo;
USE SCHEMA public;
SELECT * FROM p2p_users;
SELECT * FROM p2p_agg_transactions;

In [ ]:
CREATE OR REPLACE VIEW p2p_users_vw (nodeId) AS
    SELECT DISTINCT p2p_demo.public.p2p_users.NODEID as nodeid
    FROM p2p_users;

Unsuspend Compute Pool

In [ ]:
SHOW COMPUTE POOLS

In [ ]:
ALTER COMPUTE POOL IF EXISTS NEO4J_GRAPH_DATA_SCIENCE_POOL_CPU_X64_L RESUME

In [ ]:
call se_snow_neo4j_graph_analytics.gds.show_available_compute_pools();

In [ ]:
GRANT USAGE, MONITOR ON COMPUTE POOL NEO4J_GRAPH_DATA_SCIENCE_POOL_CPU_X64_L TO APPLICATION se_snow_neo4j_graph_analytics;

In [ ]:
CALL se_snow_neo4j_graph_analytics.gds.create_session('CPU_X64_L');


In [ ]:
GRANT SELECT ON ALL VIEWS IN SCHEMA p2p_demo.public TO APPLICATION se_snow_neo4j_graph_analytics;

In [ ]:
SELECT se_snow_neo4j_graph_analytics.gds.graph_project('g', {
    'defaultTablePrefix': 'p2p_demo.public',
    'nodeTables' : ['p2p_users_vw'],
    'relationshipTables': {
        'P2P_AGG_TRANSACTIONS': {
        'sourceTable': 'p2p_users_vw',
        'targetTable': 'p2p_users_vw'
        }
    }
});

In [ ]:
SELECT se_snow_neo4j_graph_analytics.gds.louvain('g', {'mutateProperty': 'community_id'});

In [ ]:
SELECT se_snow_neo4j_graph_analytics.gds.write_nodeproperties_to_table('g', {
  'nodeLabels': ['p2p_users_vw'],
  'nodeProperties': ['community_id'],
  'tableSuffix': '_louvain'
});

In [ ]:
SELECT se_snow_neo4j_graph_analytics.gds.graph_drop('g');

In [ ]:
CALL se_snow_neo4j_graph_analytics.gds.stop_session();

Change the Louvain results table ownership to the gds_role

In [ ]:
GRANT OWNERSHIP ON TABLE p2p_demo.public.P2P_USERS_VW_LOUVAIN TO ROLE gds_role REVOKE CURRENT GRANTS;


In [ ]:
SELECT p.NODEID, p.FRAUD_TRANSFER_FLAG, lv.COMMUNITY_ID
    FROM p2p_demo.public.P2P_USERS_VW_LOUVAIN AS lv JOIN p2p_demo.public.p2p_users AS p
        ON lv.NODEID = p.NODEID
    ORDER BY lv.COMMUNITY_ID, p.NODEID;

# 3.8

In [ ]:
CALL se_snow_neo4j_graph_analytics.internal.louvain('CPU_X64_XS', {
  'project': {
    'defaultTablePrefix': 'p2p_demo.public',
    'nodeTables': ['p2p_users'], 
    'relationshipTables': {
      'p2p_agg_transactions': {
        'sourceTable': 'p2p_users',
        'targetTable': 'p2p_users'
      }
    }
  },
  'compute': { 'consecutiveIds': true },
  'write': [
    {
      'nodeLabel': 'louvain_id', 
      'outputTable': 'p2p_demo.public.p2p_users_id'
    }
  ]
});


# 3.9

In [ ]:
SHOW COMPUTE POOLS

In [ ]:
CALL se_snow_neo4j_graph_analytics.internal.wcc('NEO4J_GRAPH_DATA_SCIENCE_POOL_CPU_X64_L', 'GDSONSNOWFLAKE', {
  'project': {
    'defaultTablePrefix': 'p2p_demo.public',  
    'nodeTables': ['p2p_users'], 
    'relationshipTables': {
      'p2p_agg_transactions': {
        'sourceTable': 'p2p_users',
        'targetTable': 'p2p_users'
      }
    }
  },
  'compute': { 'consecutiveIds': true },
  'write': [
    {'nodeLabel': 'p2p_users', 'outputTable': 'p2p_demo.public.p2p_users_wcc_id'}
  ]
});

In [ ]:
CALL se_snow_neo4j_graph_analytics.internal.wcc('CPU_X64_XS', {
  'project': {
    'defaultTablePrefix': 'p2p_demo.public',
    'nodeTables': ['p2p_users'], 
    'relationshipTables': {
      'p2p_agg_transactions': {
        'sourceTable': 'p2p_users',
        'targetTable': 'p2p_users'
      }
    }
  },
  'compute': { 'consecutiveIds': true },
  'write': [{'nodeLabel': 'p2p_users', 'outputTable': 'p2p_demo.public.p2p_users_id'}]
});

In [ ]:
CALL app.internal.louvain('CPU_X64_L', {
  'defaultTablePrefix': 'p2p_demo.public',  
  'project': {
    'nodeTables': ['p2p_users'], 
    'relationshipTables': {
      'p2p_agg_transactions': {
        'sourceTable': 'p2p_users',
        'targetTable': 'p2p_users'
      }
    }
  },
  'compute': { 'consecutiveIds': true }
  'write': [
    {'nodeLabel': 'louvain_id', 'outputTable': 'p2p_users_id'}
  ]
});
